In [64]:
import psycopg2
import pandas as pd
import os
from dotenv import load_dotenv

## Assemble Data for Endpoint

In [65]:
# retrive Demographic data from db
def demographics(zipCode):
    connection = dbConnect()
    zipData = queryZip(zipCode, connection)
    demographics = formatDemographics(zipData)
    connection.close()
    return(demographics)

In [66]:
# retrive Demographic data from db
def demographicsPct(zipCode):
    connection = dbConnect()
    zipData = queryZip(zipCode, connection)
    demographics = formatDemographicsPct(zipData)
    connection.close()
    return(demographics)

In [67]:
# Connect to DB
def dbConnect():
    try:
        load_dotenv()
        connection = psycopg2.connect(
            database = "postgres",
            user = "census",
            password = os.environ.get('PASSWORD'),
            host = os.environ.get('HOST'),
            port = 5432
            )
        return(connection)
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        connection.rollback()
        cursor.close()
        return('Error')

In [68]:
# Retrieve proper data about Zip Code
def queryZip(zipCode, connection):
    try:
        cursor = connection.cursor()
        cursor.execute("""
            SELECT * 
            FROM demographics d
            WHERE d.Zip_Code = %s;
            """,
            [zipCode,])
        zipData = cursor.fetchone()
        cursor.close()
        return(zipData)
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        connection.rollback()
        cursor.close()
        return('Error')

In [69]:
# Format Zip Code data
def formatDemographics(zipData):
    try:
        cleanZip = {'Zip Code': zipData[0],
         'Population': zipData[1],
         'Race': {'White': zipData[2],
          'Black or African American': zipData[3],
          'American Indian and Alaska Native': zipData[4],
          'Asian': zipData[5],
          'Native Hawaiian and Other Pacific Islander': zipData[6],
          'Some other race': zipData[7],
          'Two or more races': zipData[8]},
         'Hispanic': {'Hispanic or Latino origin': zipData[9],
          'Not Hispanic or Latino': zipData[10]},
         'Age': {'Under 35 years': zipData[11],
          '35 to 44 years': zipData[12],
          '45 to 54 years': zipData[13],
          '55 to 64 years': zipData[14],
          '65 to 74 years': zipData[15],
          '75 to 84 years': zipData[16],
          '85 years and over': zipData[17]},
         'Education': {'Less than high school graduate': zipData[18],
          'High school graduate': zipData[19],
          "Some college": zipData[20],
          "Bachelor's degree or higher": zipData[21]}}
        return(cleanZip)
    except:
        return('Error')

In [70]:
# Format Zip Code data and convert to pct
def formatDemographicsPct(zipData):
    try:
        zipPercent = list(zipData[1:])
        zipPercent[:] = [round(100*x/zipPercent[0],2) for x in zipPercent]
        zipPercent = [zipData[0]] + zipPercent
        cleanZip = {'Zip Code': zipPercent[0],
         'Population': zipPercent[1],
         'Race': {'White': zipPercent[2],
          'Black or African American': zipPercent[3],
          'American Indian and Alaska Native': zipPercent[4],
          'Asian': zipPercent[5],
          'Native Hawaiian and Other Pacific Islander': zipPercent[6],
          'Some other race': zipPercent[7],
          'Two or more races': zipPercent[8]},
         'Hispanic': {'Hispanic or Latino origin': zipPercent[9],
          'Not Hispanic or Latino': zipPercent[10]},
         'Age': {'Under 35 years': zipPercent[11],
          '35 to 44 years': zipPercent[12],
          '45 to 54 years': zipPercent[13],
          '55 to 64 years': zipPercent[14],
          '65 to 74 years': zipPercent[15],
          '75 to 84 years': zipPercent[16],
          '85 years and over': zipPercent[17]},
         'Education': {'Less than high school graduate': zipPercent[18],
          'High school graduate': zipPercent[19],
          "Some college": zipPercent[20],
          "Bachelor's degree or higher": zipPercent[21]}}
        return(cleanZip)
    except:
        return('Error')

In [71]:
# Endpoint Testing
zipCode = '91007'
demographicsPct(zipCode)

{'Zip Code': '91007',
 'Population': 100.0,
 'Race': {'White': 33.03,
  'Black or African American': 1.86,
  'American Indian and Alaska Native': 0.26,
  'Asian': 55.16,
  'Native Hawaiian and Other Pacific Islander': 0.34,
  'Some other race': 5.32,
  'Two or more races': 4.03},
 'Hispanic': {'Hispanic or Latino origin': 14.45,
  'Not Hispanic or Latino': 27.34},
 'Age': {'Under 35 years': 12.59,
  '35 to 44 years': 17.98,
  '45 to 54 years': 22.86,
  '55 to 64 years': 19.41,
  '65 to 74 years': 13.83,
  '75 to 84 years': 6.62,
  '85 years and over': 6.7},
 'Education': {'Less than high school graduate': 6.36,
  'High school graduate': 15.45,
  'Some college': 19.38,
  "Bachelor's degree or higher": 58.81}}